In [14]:
# nuclio: ignore
import nuclio

In [15]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

In [16]:
%%nuclio config 
kind = "nuclio"
spec.build.baseImage = "mlrun/ml-models"

%nuclio: setting kind to 'nuclio'
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


In [17]:
%nuclio cmd -c pip install git+https://github.com/v3io/grafwiz --upgrade
%nuclio cmd -c pip install v3io_frames

In [18]:
# nuclio: start-code

In [19]:
from grafwiz import *
import os
import v3io_frames as v3f

In [40]:
def handler(context,event):
    
    grafana_url = "http://grafana-dani"
    streamview_url = context.streamview_url
    readvector_url =context.readvector_url
    v3io_container = context.V3IO_CONTAINER
    
    context.logger.info("Initializing DataSources1")
    context.logger.info(streamview_url)
    context.logger.info(readvector_url)
    
    DataSource(name='Iguazio').deploy(grafana_url, use_auth=True)
    DataSource(name='stream-viewer', frames_url=streamview_url).deploy(grafana_url, use_auth=False, overwrite=False)
    DataSource(name='read-vector', frames_url=readvector_url).deploy(grafana_url, use_auth=False, overwrite=False)
    
    dash = Dashboard("stocks", start='now-7d', dataSource='Iguazio')
    
    # Add a symbol combo box (template) with data from the stocks table
    dash.template(name="SYMBOL", label="Symbol", query="fields=symbol;table=" + os.getenv('V3IO_USERNAME') + "/stocks/stocks_kv;backend=kv;container=users")

    # Create a table and log viewer in one row
    tbl = Table('Current Stocks Value', span=12).source(table=context.stocks_kv,fields=['symbol','volume', 'price', 'sentiment', 'last_updated'],container=v3io_container)
    dash.row([tbl])
    
    # Create 2 charts on the second row
    metrics_row = [Graph(metric).series(table=context.stocks_tsdb, fields=[metric], filter='symbol=="$SYMBOL"',container=v3io_container) for metric in ['price','volume']]
    metrics_row.append(Graph('sentiment').series(table=context.stocks_tsdb, fields=['sentiment'], filter='symbol=="$SYMBOL"', container=v3io_container))
    dash.row(metrics_row)
    
    log = Table('Articles Log', dataSource='stream-viewer', span=12)
    dash.row([log])

    log = Table('Vector Log', dataSource='read-vector', span=12)
    dash.row([log])
    
    dash.deploy(grafana_url)
    
    return "Done"

In [51]:
def init_context(context):
    context.logger.info("init context") 
    setattr(context, 'stocks_stream', os.getenv('STOCKS_STREAM', os.getenv('V3IO_USERNAME') + '/stocks/stocks_stream'))
    setattr(context, 'stocks_tsdb', os.getenv('STOCKS_TSDB_TABLE', os.getenv('V3IO_USERNAME') + '/stocks/stocks_tsdb'))
    setattr(context, 'stocks_kv', os.getenv('STOCKS_KV', os.getenv('V3IO_USERNAME') + '/stocks/stocks_kv'))
    setattr(context, 'V3IO_CONTAINER', os.getenv('V3IO_CONTAINER', 'users'))
    setattr(context, 'streamview_url', os.getenv('streamview_url', 'http://default-tenant.app.dev8.lab.iguazeng.com:30819'))
    setattr(context, 'readvector_url', os.getenv('readvector_url', 'http://default-tenant.app.dev8.lab.iguazeng.com:31577'))

In [52]:
# nuclio: end-code

In [ ]:
# create a test event and invoke the function locally
init_context(context)
event = nuclio.Event(body='')
resp = handler(context, event)

In [35]:
from mlrun import code_to_function
fn = code_to_function('grafana-tryout',
                      handler='handler')

In [36]:
addr = fn.deploy()

> 2021-04-04 11:22:07,377 [info] Starting remote function deploy
2021-04-04 11:22:07  (info) Deploying function
2021-04-04 11:22:07  (info) Building
2021-04-04 11:22:07  (info) Staging files and preparing base images
2021-04-04 11:22:07  (info) Building processor image
2021-04-04 11:22:12  (info) Build complete
2021-04-04 11:22:16  (info) Function deploy complete
> 2021-04-04 11:22:17,306 [info] function deployed, address=default-tenant.app.dev8.lab.iguazeng.com:30574


In [34]:
!curl {addr}

Done